# Introduction to Tensorflow

In [10]:
!activate tensorflow

In [11]:
import tensorflow as tf

**TensorFlow** is a powerful open source software library for numerical computation, particularly well suited and fine-tuned for large scale Machine Learning. 

Its basic principle is simple: you first define in python a graph of computations to perform then TensorFlow takes that graph and runs it efficiently using optimized C++ code

![Simple Computation Graph](https://pocckg.by3302.livefilestore.com/y4mfrBf3XFd5pBe7r_mAhLfldmThemf6tafYHnVXijt2og6em3rKaoZnnmtcpvlPGQB3g0v5LNesh_6iwVO2yAcFUgNhL7OHDvOAVmj8LB4ebiIZyuRjDFucZRsMiFiCNuVfc66izbBxaRzcyYyGlKgxWB4HxLQCWme5hu8fu-x6MggEfnUVWzeqbvQ20AHxB6RGCXVlzgeMXmXSKj87cKQdg?width=474&height=201&cropmode=none)

Most importantly, it is possible to break up the graph into several chunks and run
them in parallel across multiple CPUs or GPUs. TensorFlow
also supports distributed computing, so you can train colossal neural networks on
humongous training sets in a reasonable amount of time by splitting the computa‐
tions across hundreds of servers. 

TensorFlow can train a network
with millions of parameters on a training set composed of billions of instances with
millions of features each. This should come as no surprise, since TensorFlow was developed by the Google Brain team and it powers many of Google’s large scale services, such as Google Cloud Speech, Google Photos, and Google Search.

**Note**
1. TF is not limited to neural networks or even Machine Learning, you could run quantum physics simulations
if you wanted.
2. Not to be confused with the TF Learn library which is an independent project.
3. Open Sourced in November 2015.
4. Features: Has led it to boost with respect to other libraries like Theano, Caffe, Deepleaning4j, etc..
    * Clean Design
    * Scalability -- Runs on Linux, Windows, MacOSX and mobile devices
    * Flexibility
    * Documentation 
    * Priovides a python API called *TFLearn(tensorflow.contrib.learn)*  compatible with Scikit-Learn 
    * provides optimization nodes to search for the parameters that minimize a cost function
    

![](https://p4cckg.by3302.livefilestore.com/y4mu51nU1CfxMo56gTWbJUHbmgQbJmmP7KWvP-3EjXpTjagQ3eANRWDsXYsWwaSnW9CFjOFOexvX6255jtsczC8G6ZVXdiBguTfZcjYW-MLx2szoWmahJ_cXKuTQ2-qoPJmQlNotP5loPFxS0sJ2j7wnW3X1R_7xnamTyRVbiDaekLqBqIKr2uEzo3KbGC80XEiSojHIh66J5WAS3e-PXjwWw?width=453&height=336&cropmode=none)

## 1. Creating first graph and running it in a session

#### Create graph

In [1]:
import tensorflow as tf
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

#just creates computation graph, not performs any calculations

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Create Tensorflow session

To evaluate this graph you need to open a TensorFlow session and use it to initialize the variables and evaluate f. 

A TensorFlow session takes care of placing the operations onto devices such as CPUs and GPUs and running them, and it holds all the variable values


In [3]:
session = tf.Session()
session.run(x.initializer)
session.run(y.initializer)
result = session.run(f)
print(result)
session.close() # frees up resources

42


Having to repeat sess.run() all the time is a bit cumbersome, but fortunately there is
a better way:

Inside the with block, the session is set as the default session. 
Calling x.initial
izer.run() is equivalent to calling tf.get_default_session().run(x.initial
izer), and
similarly f.eval() is equivalent to calling
tf.get_default_session().run(f). This makes the code easier to read. Moreover,
the session is automatically closed at the end of the block.

In [11]:
with tf.Session() as session:
 x.initializer.run()
 y.initializer.run()
 result = f.eval()
 print(result)

42


Instead of manually running the initializer for every single variable, you can use the
initialize_all_variable() function. Note that it does not actually perform the initialization immediately, it creates a node in the graph that will initialize all variables
when it is run:

In [14]:
init = tf.global_variables_initializer() #prepare an init node

with tf.Session() as session:
    init.run() # actual initialization of all the varriables 
    result = f.eval()
    print(result)

42


Inside Jupyter or within a python shell you may prefer to create an **InteractiveSession**. ***The only difference with a regular Session is that when it is created it automatically sets itself as the default session,*** so you don’t need a with block (but you do need
to close the session manually when you are done with it):

In [15]:
session = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

session.close()

42


A Tensorflow program is split into two parts:
1. Build Computation Graph (*Construction Phase*)
2. Run Computation Graph (*execution Phase*)

The construction phase typically builds a computation graph
representing the ML model and the computations required to train it. The execution
phase generally runs a loop that evaluates a training step repeatedly (for example one
step per mini-batch) gradually improving the model parameters. 

### More on subgraphs

In [11]:
x = 2
y = 3

operation1 = tf.add(x,y)
operation2 = tf.multiply(x,y)
# it will not get evaluated 
useless = tf.multiply(x,operation1)

operation3 = tf.pow(operation2,operation1)

with tf.Session() as session:
    z = session.run(operation3)
    print(z)

7776


Here, we only wants the value of z and z does not depends on useless,
**Session will not compute the value of useless**

In [12]:
x = 2
y = 3

operation1 = tf.add(x,y)
operation2 = tf.multiply(x,y)
# it will not get evaluated 
useless = tf.multiply(x,operation1)

operation3 = tf.pow(operation2,operation1)

with tf.Session() as session:
    z, not_useless = session.run([operation3,useless])
    print(z)
    print(not_useless)

7776
10


pass all variables whose values you wants to a list in fetches 

***It is possible to break graphs into several chunks and run them parallelly across multiple CPUs, Gpus or devies ***

### Distributed Computation 

##### Get devices 

In [18]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'CPU']

In [19]:
print(get_available_gpus())

['/device:CPU:0']


In [26]:
# put part of graph on specific CPU or GPU

# Create a graph
with tf.device('/CPU:0'):
    a = tf.constant([1.0,2.0,3.0,4.0,5.0,6.0],name='a')
    b = tf.constant([1.0,2.0,3.0,4.0,5.0,6.0],name='b')
    c = tf.multiply(a,b)

# create a session with log_device_placement set to true
session = tf.Session(config=tf.ConfigProto(log_device_placement=True))

print(session.run(c))

[ 1.  4.  9. 16. 25. 36.]


## 2. Managing Graphs

    Nodes you creates gets added to default graph, But sometimes you may want to manage multiple independent graphs. 
    You can do it by creating multiple independent graphs. you can do this by creating a new Graph and temporarily 
    making it default graph inside `with block

In [27]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [50]:
g = tf.Graph()
with g.as_default():
    x2 = tf.add(3,5)
    
print(x2.graph is g)
print(x2.graph is tf.get_default_graph())

# session = tf.Session()
with tf.Session(graph=g) as sess:
    print(sess.run(x2))

True
False
8


In [46]:
del graph

In [37]:
tf.reset_default_graph()

In [ ]:
# getting handle to default graph
g = tf.get_default_graph()

### Graphs best practices
    Do not mix default graph and user created graphs

In [53]:
# ERROR PRONE
g = tf.Graph()

# Add ops to the default graph

a = tf.constant(3)
# Add ops to user created graph

with g.as_default():
    b = tf.constant(5)

In [52]:
# BETTER BUT STILL NOT GOOD ENOUGH because no more than one graph
g1 = tf.get_default_graph()
g2 = tf.Graph()

# add ops to default graph

with g1.as_default():
    a = tf.constant(3)
    
# add ops to the user created graph
with g1.as_default():
    b = tf.constant(5)

## Why Graphs
1. Save computation (only run subgraphs that lead to the values you want to fetch)
2. Break computation into small, different pices to facilitates auto-differentiation
3. Facilitate distributed computation, spread the work across multiple CPUs, GPUs or devices
4. Many common machine learning models are commonly taught and visualized as directed graphs already(NN)

### Lifecycle of Node Value

* When you evaluate a node,Tensorflow autometically determines the set of nodes that it depends on and it evaluates these nodes first.For example, consider the following code:

In [55]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as session:
    print(y.eval())
    print(z.eval())

10
15


    First, this code defines a very simple graph. Then it starts a session and runs the graph to evaluate y.
    It is important to note that it will not reuse the results from previous evaluation of w and x, 
    The preceding code evaluates w and x twice.
    * A variable start its life when its initializer is run, and it ends when the session is closed.
    If you wants to evaluate y and z efficiently, without evaluating w and x twice, you can code like:

In [56]:
with tf.Session() as session:
    y_val,z_val = session.run([y,z])
    print(y_val)
    print(z_val)

10
15


### Note:
**In single-process Tensorflow, multiple session do not share any state, even if they reuse the same graph(each session would have its own copy of every variable). In distributed Tensorflow variable state is stored on the server, not in sessions, so multiple sessions can share the same variables. **

In [ ]:
# 1. tensor variable vs normal variables 